In [ ]:
# this script calculates vorticity (Alex code) on a subset of velocity data (Vicente Code)
import datetime as dt
import matplotlib.pyplot as plt
from netCDF4 import num2date
import numpy as np
import pyart;
import glob
import pytz
from scipy.spatial import Delaunay
import cmocean
import matplotlib.lines as mlines

import cartopy.crs as ccrs 
import cartopy.feature as cfeature

from functions_radar3 import get_radar_from_aws
import SNmods3 as snmods
import probe_info
import pickle
import pandas as pd
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy.ndimage as nd
from collections import Counter

def get_loc(date):
    year = date[0:4]
    if year == '2017':
        probe_locs = probe_info.probe_locs_2017 
    else:
        probe_locs = probe_info.probe_locs_2016
    return probe_locs

#Set up our projection
crs = ccrs.PlateCarree()
globe = ccrs.Globe(datum='WGS84',ellipse='sphere')

# Get data to plot state and province boundaries
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lakes',
        scale='10m',
        facecolor='none')

# note: format of pickle files are:
# sweep 1 lon, sweep 1 lat, time of sweep, vort, velocity, sweep0 lon, sweep0 lat, reflectivity

In [ ]:
def ppi_vort(radar, swp_id=1):
    ''' updated Sept 28 2020, missing the multiplication by 2'''
    vel = radar.get_field(swp_id,'velocity')#in m/s
    phi = np.unique(radar.get_elevation(swp_id))[0]#in degrees
    theta = radar.get_azimuth(swp_id)#in degrees
   
    rangearray = np.tile(radar.range['data'],(len(theta),1))# in m
    vort = np.zeros_like(vel)
    vort=2*(1./(rangearray*np.sin(np.deg2rad(90-phi))))*np.gradient(vel,np.deg2rad(theta),axis=0)
    return vort

def get_subset(reflon, reflat, lon,lat,field):
    xs,xe = reflon-.2, reflon+.2
    ys,ye = reflat-.3,reflat+.3

    buffer_size=0
    xcoords = ([xs-buffer_size,xs-buffer_size,
                xe+buffer_size,xe+buffer_size,
                xs-buffer_size])
    ycoords = ([ys-buffer_size,ye+buffer_size,
                ye+buffer_size,ys-buffer_size,
                ys-buffer_size])
    bbox = np.asarray([xcoords,ycoords])

    bounding_box = bbox

    ######

    #draw outer boundary
    hull_bounds = Delaunay((bounding_box.T))
    #find points from data that fall within the box
    simplices   = hull_bounds.find_simplex(np.vstack((lon.flatten(),lat.flatten())).T) >= 0.
    #get indices
    ind_unravel = np.unravel_index(np.where(simplices==True)[0],shape=(lon.shape))

    # get indices and find unique values
    indx,indy = np.unique(ind_unravel[0]),np.unique(ind_unravel[1])
    yshape,xshape = indy.shape[0],indx.shape[0]


    try:
        #index the data of the coordinate and velocity fields and reshape them to a smaller size
        blon=(lon[indx.min():indx.max(),indy.min():indy.max()])
        blat=(lat[indx.min():indx.max(),indy.min():indy.max()])
        bf= (field[indx.min():indx.max(),indy.min():indy.max()])


    except:
       # if the indices go across the edge of the data, you have to rotate all the indices so 
        # they are all on one side of the 0th index
        mask = np.where((indx[1:]-indx[0:-1] > 1))[0][0] +1
        indx= np.hstack((indx[mask:]-np.shape(lon)[0],indx[:mask]))

        shift = np.max([len(indx[mask:]),len(indx[:mask]) ])
        new_indx = indx+shift
        new_indy = indy+shift

        # roll axis in both x and y directions
        new_lon = np.roll(lon, shift, axis=0)
        new_lat = np.roll(lat, shift, axis=0)
        new_field = np.roll(field, shift, axis=0)

        new_lon = np.roll(new_lon, shift, axis=1)
        new_lat = np.roll(new_lat, shift, axis=1)
        new_field = np.roll(new_field, shift, axis=1)

        #print(new_indx.min(), new_indx.max())
        #dx = 

        blon = (new_lon[new_indx.min():new_indx.max(),new_indy.min():new_indy.max()])
        blat = (new_lat[new_indx.min():new_indx.max(),new_indy.min():new_indy.max()])
        bf =   (new_field[new_indx.min():new_indx.max(),new_indy.min():new_indy.max()])
        
        
    # if subset doesn't work properly and it's all the data
    if (indy.shape[0]) > 100:
        try:
            #print('yea')
            mask = np.where((indx[1:]-indx[0:-1] > 1))[0][0] +1
            indx= np.hstack((indx[mask:]-np.shape(lon)[0],indx[:mask]))

            shift = np.max([len(indx[mask:]),len(indx[:mask]) ])
            new_indx = indx+shift
            new_indy = indy+shift

            # roll axis in both x and y directions
            new_lon = np.roll(lon, shift, axis=0)
            new_lat = np.roll(lat, shift, axis=0)
            new_field = np.roll(field, shift, axis=0)

            new_lon = np.roll(new_lon, shift, axis=1)
            new_lat = np.roll(new_lat, shift, axis=1)
            new_field = np.roll(new_field, shift, axis=1)

            #print(new_indx.min(), new_indx.max())
            #dx = 

            blon = (new_lon[new_indx.min():new_indx.max(),new_indy.min():new_indy.max()])
            blat = (new_lat[new_indx.min():new_indx.max(),new_indy.min():new_indy.max()])
            bf =   (new_field[new_indx.min():new_indx.max(),new_indy.min():new_indy.max()])
        except:
            pass

        
    return blon, blat, bf
    

In [ ]:
p_filepath = '/Users/jessmcd/Documents/MyPassport_backup/VortexSE/NewAnalyses/inferred_vort/'
snname='/Users/jessmcd/Documents/MyPassport_backup/VortexSE/NewAnalyses/ALL_EVENTS_full_thermo_R20km.csv'
All_SN = pd.read_csv(snname)
SN= All_SN.loc[All_SN['type'] != 'NR']


for sn in SN.index:
    if sn%10 ==0: # keep track of how things are running
        print(sn)
        
    ID = SN.loc[sn]['ID']
    toa = pd.to_datetime(SN.loc[sn]['TOA'])
    reflat, reflon = SN.loc[sn]['Lats'], SN.loc[sn]['Lons']
    print(toa.strftime('%Y%m%d'), ID)
    
    # pull radar object and data
    for i,time in enumerate([toa-dt.timedelta(minutes=10), toa-dt.timedelta(minutes=5), toa, toa+dt.timedelta(minutes=5), toa+dt.timedelta(minutes=10)]):
    #for i,time in enumerate([ toa, toa+dt.timedelta(minutes=5)]):
        
        print('working on time ', i)
        sweep=5
        
        # radar down on 10 March
        if toa.strftime('%m%d') == '0309' or  toa.strftime('%m%d') == '0310':
            radar_namelist, radar_list = get_radar_from_aws('KOHX', time, time)
        else:
            radar_namelist, radar_list = get_radar_from_aws('KHTX', time, time)
            
        radar = radar_list[0]
        lat, lon, _ = radar.get_gate_lat_lon_alt(sweep=sweep)
        
        # get vorticity subset (sweep=1)
        vorticity = ppi_vort(radar,swp_id=sweep)
        lon_sub, lat_sub, vort_sub = get_subset(reflon, reflat, lon, lat, vorticity)

        # get velocity subset (sweep=1)
        vel = radar.get_field(sweep,'velocity')
        _, _, vel_sub = get_subset(reflon, reflat, lon, lat, vel)
        
        # get reflectivity subset (sweep=0)
        lat, lon, _ = radar.get_gate_lat_lon_alt(sweep=sweep)
        ref = radar.get_field(sweep,'reflectivity')
        lon_subref, lat_subref, ref_sub = get_subset(reflon, reflat, lon, lat, ref)
        
        # create pickle file
        r_time = num2date(radar.time['data'][radar.sweep_start_ray_index['data'][0]], radar.time['units'])
        datapackage = np.asarray([lon_sub, lat_sub, r_time, vort_sub, vel_sub, lon_subref, lat_subref, ref_sub])
        
    
        # file name example "20160331_0215A1_2325_0"
        fname = '{}{}_{}_{}_{}_sweep5.npy'.format(p_filepath, toa.strftime('%Y%m%d'), ID, toa.strftime('%H%M'), i)
        #pickle.dump(datapackage, open(fname, 'wb'))
        
        np.save(fname, datapackage)


In [ ]:
def plot_vort(f1, f2, f3, ID, toa, reflat,reflon):

    fig = plt.figure(figsize=(15,10))
    
    gs = gridspec.GridSpec(1,3)
    gs.update(left=0.05, right=0.96, wspace=0.05, hspace=0.15)
    ax1 = plt.subplot(gs[0,0], projection=crs)
    ax2 = plt.subplot(gs[0,1], projection=crs)
    ax3 = plt.subplot(gs[0,2], projection=crs)
    
     # color bar
    gs0= gridspec.GridSpec(1,1)
    gs0.update(left=0.97, right=0.99, wspace=0.05)
    ax_cbar=plt.subplot(gs0[0])
   
    
    for ax, f in zip([ax1, ax2, ax3], [f1, f2, f3]):
        vort_plot = ax.pcolormesh(f[0],f[1], f[3], vmin=-0.015, vmax=.015, cmap =cmocean.cm.curl_r)
        ax.set_title('{}'.format(f[2].strftime('%H%M UTC')), fontsize=20)
        
        ax.scatter(reflon, reflat, s=(28**2), color='k', marker='*')
        ax.scatter(reflon, reflat, s=(18**2), color='w', marker='*')
        ax.tissot(rad_km=15, lons=reflon, lats=reflat, facecolor='none', edgecolor='gray', linewidth=2,
                 linestyle='--')
        
        
    cmap = cmocean.cm.curl_r
    norm = plt.Normalize(vmin=-0.15, vmax=.15)
    cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), pad=0.02, cax=ax_cbar)
    cbar.set_label('Inferred Vorticity', fontsize = 20, labelpad = 8)
    cbar.ax.tick_params(labelsize = 16)
#

    plt.suptitle('{} {}'.format(toa.strftime('%Y%m%d'), ID), fontsize=28)
    plt.show()
    
    
def plot_vort5(f1, f2, f3, f4, f5, ID, toa, reflat,reflon):
    ''' multiplied vorticity values by 2 on Sept 28'''

    fig = plt.figure(figsize=(20,5))
    
    gs = gridspec.GridSpec(1,5)
    gs.update(left=0.01, right=0.99, wspace=0.0, hspace=0.04)
    ax1 = plt.subplot(gs[0,0], projection=crs)
    ax2 = plt.subplot(gs[0,1], projection=crs)
    ax3 = plt.subplot(gs[0,2], projection=crs)
    ax4 = plt.subplot(gs[0,3], projection=crs)
    ax5 = plt.subplot(gs[0,4], projection=crs)
    
    
    for ax, f in zip([ax1, ax2, ax3, ax4, ax5], [f1, f2, f3, f4, f5]):
        try:
            #ax.set_aspect('auto')
            vort_plot = ax.pcolormesh(f[0],f[1], f[3], vmin=(-0.015*2), vmax=(.015*2), cmap =cmocean.cm.curl_r)
            ax.contour(f[0],f[1], f[3],levels=[0.005*2,.2], colors='k', linewidth=1)
            ax.contour(f[0],f[1], f[3],levels=[0.003*2,.2], colors='#545d6d')

            ax.text(0.03, .93, '{}'.format(f[2].strftime('%H%M UTC')), fontsize=16, transform=ax.transAxes)
            ax.scatter(reflon, reflat, s=(12**2), color='k', marker='*')
            ax.tissot(rad_km=15, lons=reflon, lats=reflat, facecolor='none', edgecolor='gray', linewidth=2,
                     linestyle='--')
        except:
            ax.text(0.03, .93, '{}'.format(f[2].strftime('%H%M UTC')), fontsize=16, transform=ax.transAxes)
            ax.scatter(reflon, reflat, s=(12**2), color='k', marker='*')
            ax.tissot(rad_km=15, lons=reflon, lats=reflat, facecolor='none', edgecolor='gray', linewidth=2,
                     linestyle='--')
#         val, edge = find_vort_max(f[3])
#         ax.contour(f[0],f[1], edge, levels =[.99,1.01], colors='k')
#         ax.text(0.02, .03, 'vort val: {}'.format(np.round(val, 7)), fontsize=18, transform=ax.transAxes,
#                weight='bold');
        
    # colorbar   
    cmap = cmocean.cm.curl_r
    norm = plt.Normalize(vmin=(-0.015*2), vmax=(.015*2))
    axins = inset_axes(ax2, width="5%",  height="100%", loc='lower left',bbox_to_anchor=(.5, -0.15, 43, .08),
               bbox_transform=ax2.transAxes,borderpad=0,)
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=axins, orientation='horizontal')
    #cbar.set_label('Inferred Vorticity', fontsize = 16, labelpad = 8)
    cbar.ax.tick_params(labelsize = 14)


    plt.suptitle('{} {} Inferred Vorticity'.format(toa.strftime('%Y%m%d'), ID), fontsize=18, weight='bold')
    
    #f = '/Users/jessmcd/Documents/MyPassport_backup/VortexSE/VorticityAnalysis/20170430_plots/'
    f='/Users/jessmcd/Documents/MyPassport_backup/VortexSE/NewAnalyses/inferred_vort/'
    name = '{}_{}_{}_vortval_sweep5.png'.format(toa.strftime('%Y%m%d'), ID, f3[2].strftime('%H%M UTC'))
    plt.savefig(f+name, bbox_inches='tight')
    plt.close()
 
    #plt.show()   

    
    

In [ ]:
p_filepath = '/Users/jessmcd/Documents/GitHub_repos/vortexse/MVstrength_analy/'

# for sn in SN.index[6:9]:
#     ID = SN.loc[sn]['ID']
#     toa = SN.loc[sn]['TOA']
#     reflat, reflon = SN.loc[sn]['Lats'], SN.loc[sn]['Lons']
    
for sn in rapid.index[5:9]:
    ID =  rapid.loc[sn]['ID']
    toa = rapid.loc[sn]['TOA']
    reflat, reflon = rapid.loc[sn]['Lats'], rapid.loc[sn]['Lons']
    
    sn_files = glob.glob('{}{}_{}*'.format(p_filepath, toa.strftime('%Y%m%d'), ID))
    sn_files = sorted(sn_files)
 
    f1 = pickle.load(open(sn_files[0],'rb'))
    f2 = pickle.load(open(sn_files[1],'rb'))
    f3 = pickle.load(open(sn_files[2],'rb'))
    
    f4 = pickle.load(open(sn_files[3],'rb'))
    f5 = pickle.load(open(sn_files[4],'rb'))
    
    #plot_vort(f1, f2, f3, ID, toa, reflat, reflon)
    plot_vort5(f1, f2, f3, f4, f5, ID, toa, reflat, reflon)
